In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key=os.getenv("GROQ_API_KEY")

In [2]:
from langchain_groq import ChatGroq
model=ChatGroq(model='llama-3.1-8b-instant',groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000019BE5CF86A0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000019BE5CF85B0>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [3]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content='Hi i am ai engineer')])

AIMessage(content="As an AI engineer, you must be working on developing and implementing artificial intelligence and machine learning models to solve real-world problems. That's a fascinating field.\n\nWhat specific areas of AI engineering are you interested in or working on? Are you more focused on:\n\n1. Natural Language Processing (NLP)?\n2. Computer Vision?\n3. Robotics?\n4. Deep Learning?\n5. Reinforcement Learning?\n6. Something else?\n\nOr perhaps you're working on a specific project or application, such as:\n\n1. Healthcare?\n2. Finance?\n3. Autonomous vehicles?\n4. Education?\n\nI'm here to chat and help if you'd like to discuss anything related to AI engineering.", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 137, 'prompt_tokens': 40, 'total_tokens': 177, 'completion_time': 0.194217607, 'prompt_time': 0.001804478, 'queue_time': 0.052402962, 'total_time': 0.196022085}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_e750f72ec9',

In [4]:
from langchain_core.messages import AIMessage
model.invoke([
    HumanMessage(content="Hi i am ai engineer"),
    AIMessage(content="As an AI engineer, you must be working on developing and implementing artificial intelligence and machine learning models to solve real-world problems. That's a fascinating field"),
    HumanMessage(content="hello what do i do?")

])

AIMessage(content="As an AI engineer, your tasks can vary depending on the specific project or company you're working with. Here are some common responsibilities:\n\n1. **Design and develop AI models**: You'll work on designing, implementing, and testing AI models using programming languages like Python, Java, or C++.\n2. **Data preprocessing**: Collecting, cleaning, and preprocessing data to prepare it for training AI models.\n3. **Model training and optimization**: Training AI models using machine learning algorithms and optimizing their performance.\n4. **Model deployment**: Deploying AI models in production environments, such as web applications or mobile apps.\n5. **Collaborate with cross-functional teams**: Working with data scientists, product managers, and software engineers to integrate AI models into products.\n6. **Troubleshooting and debugging**: Identifying and fixing issues with AI models or their deployment.\n7. **Staying up-to-date with industry trends**: Keeping curren

### Message History
we can use a message history class to wrap our model and make it stateful. This will keep track of 
inputs and output of model,and store them in datastore.Future interactions will load them messages 
and pass them into the chain as part of the input

In [6]:
from langchain_community.chat_message_histories import ChatMessageHistory #stores messages in memory list   
from langchain_core.chat_history import BaseChatMessageHistory #abstract base class for storing chat message history
from langchain_core.runnables.history import RunnableWithMessageHistory #manages chatmessagehistory for another runnable

store={}
def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

In [7]:
with_message_history=RunnableWithMessageHistory(model,get_session_history)

In [8]:
config={"configurable":{"session_id":"chat1"}}

In [9]:
with_message_history.invoke(
    [HumanMessage(content="Hi, i am Ai engineer")],
    config=config
)

AIMessage(content="Hello Ai engineer, what brings you here today? Are you working on a new project, seeking advice on a specific topic, or looking to discuss the latest advancements in the field of artificial intelligence? I'm all ears, and I'd be happy to help or engage in a conversation about AI. \n\nWhat type of AI projects are you involved with? Are you focusing on NLP, computer vision, reinforcement learning, or something else?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 88, 'prompt_tokens': 41, 'total_tokens': 129, 'completion_time': 0.15081157, 'prompt_time': 0.00217055, 'queue_time': 0.05479311, 'total_time': 0.15298212}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_1151d4f23c', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--1177b8f8-6214-4546-a516-f2712f3c554f-0', usage_metadata={'input_tokens': 41, 'output_tokens': 88, 'total_tokens': 129})

In [17]:
response=with_message_history.invoke(
    [HumanMessage(content="who am i?")],
    config=config
)
response.text

'You are an AI engineer.'

### Prompt Templates
prompt template help to turn raw information into a format that llm can work with.

In [18]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ('system','you are helpful assistant. Answer all the question'),
        MessagesPlaceholder(variable_name="messages")
    ]
)
chain=prompt|model

In [19]:
chain.invoke({'messages':[HumanMessage(content="hi my name is sushant")]})

AIMessage(content="Nice to meet you, Sushant. I'm happy to assist you with any questions or information you may need. How's your day going so far?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 51, 'total_tokens': 84, 'completion_time': 0.048269065, 'prompt_time': 0.002460466, 'queue_time': 0.050687963, 'total_time': 0.050729531}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_f757f4b0bf', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--08e537bd-f2ec-4d1a-8a57-41c58a12ace9-0', usage_metadata={'input_tokens': 51, 'output_tokens': 33, 'total_tokens': 84})

In [20]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [22]:
config={'configurable':{'session_id':'chat3'}}
response=with_message_history.invoke([
    HumanMessage(content='hi my name is sushant')
],config=config)
response.text

"Nice to meet you, Sushant! It's a pleasure to chat with you again. How's your day going so far?"

In [23]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

### Managing the conversation history
when building chatbots, need to manage conversation history.If left unmanaged, the list of messages
will grow unbounded and potentially overflow the context window of LLM.Therfore, it is important to
add a step to limits the size of the messages you are passing in.

In [33]:
#trim_messages=helps to reduce how many msg we're sending to the model (reduce size of chat history)
from langchain_core.messages import SystemMessage,trim_messages
trimmer=trim_messages(
    max_tokens=45,
    strategy='last',
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)
messages=[
    SystemMessage(content='You ara a good assistant.'),
    HumanMessage(content='Hi! i am vijay'),
    AIMessage(content='Hi!'),
    HumanMessage(content='I like icecream'),
    AIMessage(content='nice'),
    HumanMessage(content='what is 2+2'),
    AIMessage(content='4'),
    HumanMessage(content='thanks'),
    AIMessage(content='no problem!')
]
trimmer.invoke(messages)

[SystemMessage(content='You ara a good assistant.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like icecream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='what is 2+2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={})]

In [34]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough
chain=(
    RunnablePassthrough.assign(messages=itemgetter('messages')|trimmer)
    |prompt
    |model
)

response=chain.invoke({
    "messages":messages + [HumanMessage(content="which food do i like?")],
    "language":"English",
})
response.content

'You mentioned earlier that you like ice cream!'